This file serves as a class to form the serial connection to an Alicat device.

In [1]:
try:
    import serial
except ImportError:
    print("An error occurred while attempting to import the pyserial backend. Please check your installation and try again.")


In [2]:
class Serial_Connection(object):
    """The serial connection object from which other classes inherit port and baud settings as
    well as reading and writing capabilities. Preprocessing for GP firmware outputs is performed
    using the remove_characters function"""
    
    # A dictionary storing open serial ports for comparison with new connections being opened
    open_ports = {}
    
    def __init__(self, port='/dev/ttyUSB0', baud=19200):
        
        self.port, self.baud = port, baud
        self.devices = {}
        
        # Checks for the existence of an identical port to avoid multiple instances
        # creates a new one if none exist
        if port in Serial_Connection.open_ports:
            self.connection = Serial_Connection.open_ports[port]
        else:
            self.connection = serial.Serial(port, baud, timeout=2.0)
            Serial_Connection.open_ports[port] = self.connection
        
        self.open = True
        
    def _test_open(self):
        
        if not self.open:
            raise IOError(f"The connection to Alicats on port {self.port} not open")
    
    
    def flush(self):
        """Deletes all characters in the read/write buffer to avoid double messages or rewrites"""
        self._test_open()
        
        self.connection.flush()
        self.connection.flushInput()
        self.connection.flushOutput()

    def close(self):
        """Checks if the instance exists, clears the buffer, closes the connection, and removes
        itself from the dictionary of active ports"""
        if not self.open:
            return
        
        self.flush()
        
        self.connection.close()
        Serial_Connection.open_ports.pop(self.port, None)
        
        self.open = False
        
    def read(self):
        """Fast read method using byte arrays with a carriage return delimiter between lines.
        ~30x faster than a readline operation by reading individual characters and appending 
        them to the byte array. The byte array is then decoded into a string and returned"""
        self._test_open()
        
        line = bytearray()
        while True:
            c = self.connection.read(1)
            if c:
                line += c
                if line[-1] == ord('\r'):
                    break
            else:
                break
        
        return line.decode('ascii').strip()
    
    def write(self, ID, command):
        """Writes the input command for a device with the stated ID by encoding to ascii
        and sending through the serial connection write command. Reads out the return parsing 
        multiple lines until the buffer is clear."""
        command = str(ID) + str(command) + '\r'
        command = command.encode('ascii')
        self.connection.write(command)
        response = []
        response.append(self.read())
        while True:
            if response[-1] != '':
                response.append(self.read())
            else:
                return response[:-1]
            
    def remove_characters(self, array, **args):
        """Removes any set of string characters from an array in place. Used for
        clearing the '/x08' string from arrays retruned from the read and write commands"""
        
        if isinstance(array[0], list):
            for i in range(len(args)):
                for j in range(len(array)):
                    array[j] = list(filter((args[i]).__ne__, array[j-1]))
        else:
            for i in range(len(kwargs)):
                array = list(filter((args[i]).__ne__, array))
        return array
    
    
    
    

In [3]:
class MassFlowMeter(Serial_Connection):
    """Base class which obtains relevant gas"""
    def __init__(self, ID :str ='A', port : str ='/dev/ttyUSB0', baud : int =19200):
        super().__init__(port, baud)
        self.ID, self.port, self.baud = ID, port, baud
        self.fetch_gas_list()
        self.fetch_firmware_version()
        self.data_format()
    
    def fetch_gas_list(self):
        self.gas_list = {}
        gases = [i.split() for i in self.write(self.ID, '??G*')]
        gases = self.remove_characters(gases, '\x08')
        for i in range(len(gases)):
            self.gas_list[gases[i-1][2]] = (gases[i-1][1])[1:]
        
    
    def fetch_firmware_version(self):
        manufacturer_data = [i.split() for i in self.write(self.ID, '??M*')]
        self.manufacturer_data = self.remove_characters(manufacturer_data,'\x08')
        self.firmware_version = self.manufacturer_data[-1][-1]
        
    def data_format(self):
        self.data = []
        outputs = [i.split() for i in self.write(self.ID,'??D*')]
        outputs = self.remove_characters(outputs,'\x08')
        for i in range(2,len(outputs)):
            if outputs[i][3] != 'string' and outputs[i][2] != 'Gas':
                self.data[i-2] = [outputs[i][1][1:],outputs[i][2],outputs[i][4:]]
        
        
        

In [63]:
a = [[1,2,3,4],[2,3,4],[3,5,6,6]]
a[2][1]

5

In [43]:
AAA = Serial_Connection('COM5',19200)

In [8]:
BBB.write('A','')

['A +013.66 +030.28 -0.0000 -0.0000 +0.0000 +0000.00336 +000000.0037    Air']

In [98]:
BBB = MassFlowMeter('A','COM5',19200)

IndexError: list assignment index out of range

In [93]:
BBB.outputs[10][1]

'D10'

In [86]:
a = [1,1,1,1,1,1,1,1,1,1,1,1,1]

In [94]:
for i in range(2,len(a)):
    print(i)

2
3
4
5
6
7
8
9
10
11
12


In [39]:
BBB.firmware_version


'GP07R80'

In [40]:
BBB.gas_list

{'Air': '00',
 'Ar': '01',
 'CH4': '02',
 'CO': '03',
 'CO2': '04',
 'C2H6': '05',
 'H2': '06',
 'He': '07',
 'N2': '08',
 'N2O': '09',
 'Ne': '10',
 'O2': '11',
 'C3H8': '12',
 'n-C4H10': '13',
 'C2H2': '14',
 'C2H4': '15',
 'i-C4H10': '16',
 'Krypton': '17',
 'Xe': '18',
 'SF6': '19',
 'C-25': '20',
 'C-10': '21',
 'C-8': '22',
 'C-2': '23',
 'C-75': '24',
 'A-75': '25',
 'A-25': '26',
 'A1025': '27',
 'Star29': '28',
 'P-5': '29'}

In [68]:
BBB.flush()
BBB.close()

OSError: The connection to Alicats on port COM5 not open

In [47]:
AAA.flush()
AAA.close()

In [1]:
a = 1.51

In [2]:
a=int(a)

In [3]:
a

1